In [14]:
import numpy as np
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [15]:
# Diabetes Class

class Diabetes:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        
        # 2층 hidden layer unit 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_nodes)  
        self.b2 = np.random.rand(hidden_nodes)
        
        # 3층 output layer unit : 1 개 
        self.W3 = np.random.rand(hidden_nodes,output_nodes)
        self.b3 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        print("Diabetes object is created !!!")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z1 = np.dot(self.input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) + self.b3
        y = sigmoid(z2)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z1 = np.dot(self.input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) + self.b3
        y = sigmoid(z2)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, input_data):    
        
        z1 = np.dot(input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) + self.b3
        y = sigmoid(z2)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    # accuracy method 1
    def accuracy1(self, input_data, target_data):
        
        matched_list = []
        not_matched_list = []
        index_label_prediction_list = []
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
            index_label_prediction_list.append([index,target_data[index],logical_val])    
            
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list, index_label_prediction_list
    
    # accuracy method 2
    def accuracy2(self, test_data):
        
        matched_list = []
        not_matched_list = []
                
        input_data = test_data[ :, 0:-1 ]
        target_data = test_data[ :, -1 ]
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
    
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)

In [16]:
load_data = np.loadtxt('./diabetes (1).csv',delimiter=',')

total_xdata = load_data[:,0:-1]
total_tdata = load_data[:,[-1]]
# training data 


In [17]:
print(load_data.shape[0] * 0.7)
x_data = load_data[:int(load_data.shape[0]*0.7),0:-1]
t_data = load_data[:int(load_data.shape[0]*0.7),[-1]]
test_x_data = load_data[int(load_data.shape[0]*0.7):,0:-1]
test_t_data = load_data[int(load_data.shape[0]*0.7):,[-1]]
test_data = load_data[int(load_data.shape[0]*0.7):,:]
training_data = load_data[:int(load_data.shape[0]*0.7),:]
print("x_data.ndim = ", x_data.ndim,", x_data.shape = ", x_data.shape)
print("t_data.ndim = ", t_data.ndim,", t_data.shape = ", t_data.shape)
print("test_x_data.ndim = ", test_x_data.ndim,", test_x_data.shape = ", test_x_data.shape)
print("test_t_data.ndim = ", test_t_data.ndim,", test_t_data.shape = ", test_t_data.shape)
np.savetxt('./test_data.csv',test_data,delimiter = ',')
np.savetxt('./training_data.csv',training_data,delimiter = ',')

531.3
x_data.ndim =  2 , x_data.shape =  (531, 8)
t_data.ndim =  2 , t_data.shape =  (531, 1)
test_x_data.ndim =  2 , test_x_data.shape =  (228, 8)
test_t_data.ndim =  2 , test_t_data.shape =  (228, 1)


In [18]:
# training data 
training_data = np.loadtxt('./training_data.csv', delimiter=',',dtype = np.float32)
#print("training_data = ", training_data.shape)

#hyper-parameter
i_nodes = 8    # input nodes 개수
h1_nodes = 10  # hidden nodes 개수
o_nodes = 1    # output nodes 개수
lr = 1e-2      # learning rate
epochs = 50   # 반복횟수

# Diabetes 객체 생성
obj = Diabetes(i_nodes, h1_nodes, o_nodes, lr)

print("Neural Network Learning using Numerical Derivative...")

start_time = datetime.now()

for step in range(epochs):
    
    for index in range(len(training_data)):
        
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        obj.train(input_data, target_data)
        
    if (step % 5 == 0):
        print("epochs = ", step, "loss value = ", obj.loss_val())

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

Diabetes object is created !!!
Neural Network Learning using Numerical Derivative...
epochs =  0 loss value =  0.3357524534402149
epochs =  5 loss value =  0.40544235781644444
epochs =  10 loss value =  0.43032354768482955
epochs =  15 loss value =  0.47498453881436226
epochs =  20 loss value =  0.5385274105277263
epochs =  25 loss value =  0.6077835890681008
epochs =  30 loss value =  0.6691096353507288
epochs =  35 loss value =  0.7157947753028845
epochs =  40 loss value =  0.7476593902548853
epochs =  45 loss value =  0.7676304916633545

Elapsed Time =>  0:01:31.504595


In [19]:
test_data = np.loadtxt('./test_data.csv', delimiter=',')
print("test_data.shape = ", test_data.shape)

test_input_data = test_data[ :, 0:-1 ]
test_target_data = test_data[ :, -1 ]

(true_list, false_list, index_label_prediction_list) = obj.accuracy1(test_input_data, test_target_data)

(true_list, false_list) = obj.accuracy2(test_data)

test_data.shape =  (228, 9)
Accuracy =>  0.7719298245614035
Accuracy =>  0.7719298245614035


In [20]:
print(index_label_prediction_list)

[[0, 0.0, 1], [1, 0.0, 1], [2, 0.0, 1], [3, 0.0, 1], [4, 1.0, 1], [5, 1.0, 1], [6, 0.0, 0], [7, 0.0, 0], [8, 1.0, 1], [9, 1.0, 1], [10, 1.0, 0], [11, 1.0, 1], [12, 1.0, 1], [13, 1.0, 1], [14, 1.0, 1], [15, 1.0, 1], [16, 1.0, 1], [17, 1.0, 1], [18, 1.0, 1], [19, 1.0, 1], [20, 1.0, 1], [21, 0.0, 1], [22, 0.0, 0], [23, 1.0, 1], [24, 1.0, 1], [25, 1.0, 1], [26, 1.0, 1], [27, 1.0, 1], [28, 1.0, 1], [29, 1.0, 1], [30, 0.0, 1], [31, 1.0, 1], [32, 1.0, 1], [33, 1.0, 1], [34, 1.0, 1], [35, 1.0, 1], [36, 1.0, 1], [37, 1.0, 1], [38, 0.0, 1], [39, 1.0, 1], [40, 0.0, 0], [41, 0.0, 0], [42, 1.0, 1], [43, 1.0, 1], [44, 1.0, 1], [45, 0.0, 1], [46, 1.0, 1], [47, 0.0, 1], [48, 1.0, 1], [49, 0.0, 0], [50, 1.0, 1], [51, 0.0, 0], [52, 1.0, 1], [53, 0.0, 1], [54, 1.0, 1], [55, 1.0, 1], [56, 0.0, 0], [57, 1.0, 1], [58, 1.0, 1], [59, 0.0, 0], [60, 1.0, 1], [61, 1.0, 1], [62, 1.0, 1], [63, 1.0, 1], [64, 0.0, 0], [65, 0.0, 0], [66, 1.0, 1], [67, 0.0, 0], [68, 1.0, 1], [69, 1.0, 0], [70, 1.0, 1], [71, 1.0, 1], [